# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,college,87.3229,-154.6111,263.55,93,100,3.91,1715275749
1,1,mili,4.7367,171.5723,301.49,80,66,5.06,1715275750
2,2,port alfred,-64.1194,39.6399,269.37,85,100,12.52,1715275751
3,3,blackmans bay,-72.2757,125.1142,215.81,100,52,5.03,1715275752
4,4,nishinoomote,29.0990,133.7714,292.36,61,100,3.94,1715275752


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_city=city_data_df.hvplot.points("Lng","Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    size = "Humidity",
    color = "City"  
)
# YOUR CODE HERE

# Display the map
map_plot_city
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
results_city=city_data_df[(city_data_df['Humidity'] < 70) & 
                                  (city_data_df['Cloudiness'] == 0) & 
                                  (city_data_df['Max Temp'] <= 25) & 
                                  (city_data_df['Max Temp'] >=15)]
# YOUR CODE HERE

# Drop any rows with null values
results_city_null_values= results_city.dropna()
# YOUR CODE HERE

# Display sample data
results_city_null_values
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=results_city[['City', 'Lat', 'Lng', 'Humidity']].copy()
# YOUR CODE HERE 'Country'

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''
# YOUR CODE HERE

# Display sample data
hotel_df
# YOUR CODE HERE

,City,Lat,Lng,Humidity,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 1000# YOUR CODE HERE
params = {"apiKey": geoapify_key,
    "limit":20,
    'categories':'accommodation.hotel'
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"# YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}"# YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)# YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json()# YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


,City,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_city2 = hotel_df.hvplot.points("Lng","Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height =600,
    size = "Humidity",
    color = "City",
    hover_cols = ['City', 'Country', 'Hotel Name'] 
)
# YOUR CODE HERE

# Display the map
map_plot_city2
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)